## Reference 
   - https://python3.wannaphong.com/2018/12/named-entity-recognition-ner-pythainlp.html

In [9]:
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re
import os
import pickle
import dill

In [10]:
#จัดการประโยคซ้ำ
def Unique(p):
    text=re.sub("<^[>]*>","",p)
    text=re.sub("\[(.*?)\]","",text)
    text=re.sub("\[\/(.*?)\]","",text)
    if text not in data_not:
        data_not.append(text)
        return True
    else:
        return False

# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
    """
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
    lines = []
    str_line = ""
    with open(fileopen, 'r',encoding='utf-8-sig') as f:
        for i, line in enumerate(f.readlines()):
            if(line == "\n"):
                lines.append(str_line)
                lines[-1] = lines[-1].replace("\n"," ")
                str_line = ""
            else:
                str_line += line
        lines.append(str_line.replace("\n"," "))
    return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน

def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll
    

# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'  #เราจะไปเดินเล่นที่\tO\nหนองคาย\tB-location\n
        i+=1
    return text


# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
    text=text.strip()
    text=re.sub("<[^>]*>","",text)
    text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
    text=re.sub("(\[\w+\])","***\\1",text)
    text2=[]
    for i in text.split('***'):
        if "[" in i:
            text2.append(i)
        else:
            text2.append("[word]"+i+"[/word]")
    text="".join(text2)#re.sub("[word][/word]","","".join(text2))
    return text.replace("[word][/word]","")

# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')

# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text) # นำไปใส่ tag [word]
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
    j=0
    conll2002="" # ประกาศตัวแปรเก็บ conll2002
    for tagopen,text,tagclose in tag: # ลูปใน tag โดยเป็น (tagopen,text,tagclose)
        word_cut=word_tokenize(text,engine="newmm") # ใช้ตัวตัดคำ newmm ของ PyThaiNLP
        i=0
        txt5=""
        while i<len(word_cut): #ลูปตามจำนวน token ที่ตัดในtag
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen 
            else: # เป็น [word]
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n' #ลาว\t-location\n
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002) # เพิ่ม postag ใส่

def alldata_list(lists, pos = True):
    data_all=[]
    for data in lists:        
        data_num=[]
        try:
            txt=text2conll2002(data,pos=pos).split('\n') # นำไปแปลงเป็น conll2002
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2])) 
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
#     print(data_all)
    return data_all

In [11]:
path = "../dataset/tagged_data/"
data_not=[]
data1  = []

for name_folder in os.listdir(path):
    for f_name in os.listdir(path+name_folder):
        data1 += getall(get_data(path+name_folder+"/"+f_name))
        
datatofile_ = alldata_list(data1, pos=False)
datatofile_pos = alldata_list(data1, pos=True) # นำไปผ่านขั้นตอน 1 2 3 4
datatofile_pos[0]

[('ใน', 'RPRE', 'O'),
 ('บันทึก', 'NCMN', 'O'),
 ('ย่อ', 'VATT', 'B-d'),
 ('ที่', 'PREL', 'O'),
 ('น่าเศร้า', 'VSTA', 'B-e'),
 ('เมื่อ', 'JSBR', 'O'),
 ('วันที่', 'NCMN', 'O'),
 (' ', 'PUNC', 'O'),
 ('15', 'DCNM', 'O'),
 (' ', 'PUNC', 'O'),
 ('กันยายน', 'NCMN', 'O'),
 (' ', 'PUNC', 'O'),
 ('2547', 'NCNM', 'O'),
 (' ', 'PUNC', 'O'),
 ('จอห์น', 'VACT', 'O'),
 ('นี่', 'NCMN', 'O'),
 ('รา', 'NCMN', 'B-e'),
 ('โมน', 'NCMN', 'I-e'),
 (' ', 'PUNC', 'O'),
 ('(', 'PUNC', 'O'),
 ('ซึ่ง', 'JSBR', 'B-e'),
 ('แสดง', 'VACT', 'I-e'),
 ('ใน', 'RPRE', 'O'),
 ('ตอนนี้', 'NCMN', 'O'),
 ('กับ', 'RPRE', 'O'),
 ('รา', 'NCMN', 'O'),
 ('โมน', 'NCMN', 'O'),
 ('ส์)', 'NCMN', 'O'),
 (' ', 'PUNC', 'O'),
 ('เสียชีวิต', 'NCMN', 'O')]

In [12]:
def clear_empty_list(datatofile):
    temp_data = []
    for data in datatofile:
        if data != []:
            temp_data.append(data)
    return temp_data

In [13]:
datatofile_ = clear_empty_list(datatofile_)
datatofile_pos = clear_empty_list(datatofile_pos)

# # save data for train (CRF, LSTM, BiLSTM, BiLSTM-CRF, Bert(transformers))

In [14]:
path = "../dataset/CoNLL2002-dataset/"
file_name="biased"

with open(path+file_name+"-pos.conll","w", encoding="utf-8-sig") as f:
    i=0
    while i<len(datatofile_pos):
        for j in datatofile_pos[i]:
            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")
        if i+1<len(datatofile_pos):
            f.write("\n")
        i+=1

with open(path+file_name+".conll","w",  encoding="utf-8-sig") as f:
    i=0
    while i<len(datatofile_):
        for j in datatofile_[i]:
            f.write(j[0]+"\t"+j[1]+"\n")
        if i+1<len(datatofile_):
            f.write("\n")
        i+=1

In [15]:
path = "../dataset/data/"

pickle.dump(datatofile_pos, open(path+"biased-pos.data", 'wb'))
pickle.dump(datatofile_, open(path+"biased.data", 'wb'))
print(len(datatofile_pos))
print(datatofile_pos[0])

4999
[('ใน', 'RPRE', 'O'), ('บันทึก', 'NCMN', 'O'), ('ย่อ', 'VATT', 'B-d'), ('ที่', 'PREL', 'O'), ('น่าเศร้า', 'VSTA', 'B-e'), ('เมื่อ', 'JSBR', 'O'), ('วันที่', 'NCMN', 'O'), (' ', 'PUNC', 'O'), ('15', 'DCNM', 'O'), (' ', 'PUNC', 'O'), ('กันยายน', 'NCMN', 'O'), (' ', 'PUNC', 'O'), ('2547', 'NCNM', 'O'), (' ', 'PUNC', 'O'), ('จอห์น', 'VACT', 'O'), ('นี่', 'NCMN', 'O'), ('รา', 'NCMN', 'B-e'), ('โมน', 'NCMN', 'I-e'), (' ', 'PUNC', 'O'), ('(', 'PUNC', 'O'), ('ซึ่ง', 'JSBR', 'B-e'), ('แสดง', 'VACT', 'I-e'), ('ใน', 'RPRE', 'O'), ('ตอนนี้', 'NCMN', 'O'), ('กับ', 'RPRE', 'O'), ('รา', 'NCMN', 'O'), ('โมน', 'NCMN', 'O'), ('ส์)', 'NCMN', 'O'), (' ', 'PUNC', 'O'), ('เสียชีวิต', 'NCMN', 'O')]


# save data for train bert lazy(simpletranformers)

In [16]:
from sklearn.model_selection import train_test_split

tagged_sents = []
for data in datatofile_:
    text_inside = []
    for word, label in data:
        if word.strip() == '':
            text_inside.append(('_', label))
        else:
            text_inside.append((word, label))
    tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)

print(len(train_sents))
print(len(test_sents))

3999
1000


In [17]:
def convert_format_label(datatofile):
    data_file_bert = []
    for data in datatofile:
        list_word_lable = []
        for word_label in data:
            word = word_label[0]
            label = word_label[1]
            label = label.upper().replace("-", "_")
            list_word_lable.append((word, label))
        data_file_bert.append(list_word_lable)
    return data_file_bert

In [18]:
data_train = convert_format_label(train_sents)
data_test = convert_format_label(test_sents)
print(data_train[0])

[('Michael', 'O'), ('_', 'O'), ('McDowell', 'O'), ('_', 'O'), ('เกิด', 'O'), ('ที่', 'O'), ('ดับ', 'O'), ('ลิ', 'O'), ('น', 'O'), ('ประเทศ', 'O'), ('ไอร์แลนด์', 'O'), ('และ', 'O'), ('ได้รับ', 'O'), ('การศึกษา', 'O'), ('ที่', 'O'), ('Elite', 'B_E'), ('_', 'I_E'), ('Private', 'I_E'), ('_', 'I_E'), ('Jesuit', 'I_E'), ('_', 'I_E'), ('School', 'I_E'), ('_', 'O'), ('Gonzaga', 'O'), ('_', 'O'), ('College', 'O'), ('_', 'O'), ('และ', 'O'), ('จากนั้น', 'O'), ('ที่', 'O'), ('_', 'O'), ('University', 'O'), ('_', 'O'), ('College', 'O'), ('_', 'O'), ('Dublin', 'O'), ('_', 'O'), ('และ', 'O'), ('_', 'O'), ('King', 'O'), ("'", 'O'), ('s', 'O'), ('_', 'O'), ('Inns', 'O'), ('_', 'O'), ('ใน', 'O'), ('ดับ', 'O'), ('ลิ', 'O'), ('น', 'O'), ('ซึ่ง', 'O'), ('เขา', 'O'), ('มี', 'O'), ('คุณสมบัติ', 'O'), ('เป็น', 'O'), ('ทนายความ', 'O')]


In [19]:
# Train
path = "../dataset/CoNLL2002-dataset/"
file_name="train"

with open(path+file_name+"_CONLL_BERT.txt","w",  encoding="utf-8-sig") as f:
    i=0
    while i<len(data_train):
        for j in data_train[i]:
            f.write(j[0]+" "+j[1]+"\n")
        if i+1<len(data_train):
            f.write("\n")
        i+=1

In [20]:
# Test
path = "../dataset/CoNLL2002-dataset/"
file_name="test"

with open(path+file_name+"_CONLL_BERT.txt","w",  encoding="utf-8-sig") as f:
    i=0
    while i<len(data_test):
        for j in data_test[i]:
            f.write(j[0]+" "+j[1]+"\n")
        if i+1<len(data_test):
            f.write("\n")
        i+=1

# load data

In [21]:
path = "../dataset/data/"

with open(path+"biased-pos.data", 'rb') as file:
    datatofile = dill.load(file)

len(datatofile)

4999